In [1]:
import numpy as np
import os
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from modules.md_c_interface import launch_simulation, launch_on_doe

from time import time, sleep

from modules.md_utils import complementaire # utils
from modules.md_utils import plot_crystal, plot_2d_simu # plotting
from modules.md_utils import process_results

## Introduction 
Ions should be selected amongst the ones available [here](../md2_notebook_resources/available_ion.png).

As for the parameters, a lots of them are available. You can see them all in [1](../md2_notebook_resources/table_key_description_1.png), [2](../md2_notebook_resources/table_key_description_2.png) and [3](../md2_notebook_resources/table_key_description_3.png).

For simplicity reasons, only a few are available in the following algorithms : ion, ionE, Tset, ionT, ionP, tau, n, dt, dtv and i1.

Which should be given for example like :


## Interest

Understanding how energy absorption, reaction probability and output angle $\phi$ and $\theta$,change with:
- incoming energy;
- collision angle.

In the end, it would be necessary to compare it for various particle types (mass, charge, size, degrees of freedom etc.) in order to have a comprehensive approach of the issue. 

Also, the **type of the surface** we collide with can change. The MD-code already proposes a *SI* surface (which is the most common one). However, in the end, it would be good to be able to add *Al*.

Eventually a study of the **evolution of the surface** could be done : the law governing the output energy, angle and reaction probability ought to change with the **state of the surface**.

## Experience plan

Draw a 3D-plan of experience (on *ionE*, *ionT*). Everything else is kept equals. The thermostat temperature is choosen to be $300$ K (temperature of the neutral gas in the chamber). The azimutal angle is for a first study kept equal to 0.

Note : the simulation in the DSMC code are 2D. The azimutal angle does not make sense in regards to this code. However, it is important to either verify that is does not change the dynamic of the ion or average over many azimutal angle (which would be the most rigorous approach but also the longer one).

For each of those values, we will compute the output polar angle and the ouput energy. 

Also, we are interested in the ionization energy (cf. [wiki - Iodine](https://en.wikipedia.org/wiki/Iodine)). Let's say we launch a ion $[I^-]$. We want to know the probability for it to lose its additional electron and exits the crystal as a neutral atom. 

We see for example, that the first ionization energy for iodine is for an energy of **1008.4** kJ/mol (I did not find for $[I^-]$, but let's say for now it is the same, even though it is not). This amounts to an external energy of :  .

So if the external potential energy grows higher than this number, we can consider the ion lost it electron.

Issue one : we can not simulate $[I]$ apparenty - it does not work (like the particle goes trough the crystal and that's it).

### Notice

One of the issue we have is that Iodine is very massive (127u) while Si is only 28u. Thus, Iodine manages to come in the crystal and never leaves. 

What should be done instead is having Molybdene (95u). However, we can not really change it in the code as it is hard coded and all.

Thus, what we we'll do is maintain the "ratio" :
- Ratio $m_{molyb}/m_{si} = 3.4$
- Ratio $m_{I}/m_{Ar, simu} = 3.4$

Thus, $m_{Ar, simu}  = 37$ u roughly.

In [2]:
speed_range = [0, 25000]# possible speed range that we are interested into
polar_angle_range = [0, 90] # in degree

# converting 

def convert_to_ev(energy_per_mol): # from J/mol -> energy for one particle 
    return energy_per_mol/(1.6e-19*6.02e23)

energy_first_ionization = convert_to_ev(1008.4e3)

### Available particles

In [3]:
colors = {
    'Si':'gray',
    'Ar':'darkviolet',
    'I':'violet',
    'Kr':'lightskyblue',
    'Xe':'dodgerblue',
    'CF3':'red'
}
radii = {
    'Si': 1.11, # Angstrom
    'Ar' : 1.06,
    'I' : 1.98,
    'Kr': 2.02,
    'Xe': 2.17,
    'CF3': 1
}

### Simulations parameters

- We start with *Argon* $[Ar]$, which was modified in the code to behave like *Iodine* $[I]$. 

In [4]:
ion = 'Ar'
test_id = 'doe_1'
name = '../md2_results/test_{}_{}'.format(ion, test_id)
    
params = {
    '-ion' : ion,
    '-ionE' : 10, # eV - input energy for the ion
    '-Tset' : 300, # K - thermostat temp
    '-ionT' : 20, # Degree - phi incident - polar angle
    '-ionP' : 0, # Degree - theta incident - azimutal angle
    '-tau' : 0.01, # s3
    '-n' : 5000, # nb time steps
    '-dt' : 1.e-3, # time step, in ps
    '-i1' : 1, # number of ions launched
    '-ionCOMr' : '0.0 0.0 14.0', # ion center of mass
    # '-iui' : 1.e-3, # ion update interval
}

flags = ['+dtv']

### DOE

In [5]:
from SALib.sample import saltelli
from SALib.analyze import sobol
from SALib.test_functions import Ishigami
import numpy as np


In [6]:
keys = ['-ionE', '-ionT', '-ionP']

e = 1.6e-19
mass = 127*1.6e-27 # Iodine roughly
ratio_mass = 3.4
v_min, v_max = 10, 3e4 # from 1 to 30k m/s
nrj_ev_min, nrj_ev_max = 0.5*mass*v_min**2/(e*ratio_mass), 0.5*mass*v_max**2/(e*ratio_mass)
print(nrj_ev_min, nrj_ev_max)

In [7]:
problem = {
    'num_vars': 3,
    'names': ['Ei', 'Ti','Pi'],
    'bounds': [[nrj_ev_min, nrj_ev_max], # eV
                [-85, 85],
                [-85, 85]]
}

In [8]:
param_values = saltelli.sample(problem, 100) # 60, 2
print(param_values.shape)

array([[ 144.03808287,   38.49372589,   89.26118607],
       [   3.25266099,  -61.20590964,  163.35703776],
       [  73.08744629,  -39.58678326,    1.74355228],
       [ 108.20164083,   30.99568716,  -64.7283291 ],
       [  58.21288691,   82.5093106 , -143.29132903],
       [ 165.99194755,  -31.90132632, -147.56747297],
       [  31.11617373,   59.22922287,  122.25779251],
       [  38.81178466,  -13.6966236 ,  -77.0711782 ],
       [  93.82217125,  -75.55453894,   -3.35980348],
       [ 123.13994053,    5.74177588,   42.66642953]])

In [ ]:
np.save('param_values_analysis.npy',param_values)

### Launching simulation

In [ ]:
# launch_simulation(name, params, flags, launches = 1, verbose = True)
launch_on_doe(name, params, flags, keys = keys, design_of_experiments = doe, clean = True) # actually we are only interested in ionE and ionT

## Processing results 
The simulation can take a little time even though the previous cell is marked as completed. You should wait one or two minutes (and reload *results* if you see that all data was not yet written to disk).

In [ ]:
ions_dict = {
    'low_memory' : True,
    'crystal_height' : 12, # Angstrom
}

results = process_results(Path.cwd()/'md2_sources'/name, process_log = False, args_ions = ions_dict) # 10 ~ angstrom

In [ ]:
ions_dict = {
    'low_memory' : True,
    'crystal_height' : 12, # Angstrom
}

results = process_results(Path.cwd()/'md2_sources'/name, process_log = False, args_ions = ions_dict) # 10 ~ angstrom

In [ ]:
angles, energies = get_ground_truth(results.ion.angles, results.ion.nrjs, list_names)

In [ ]:
results.ion.get_output_angles()

In [ ]:
np.save('angles_analysis.npy',angles);
np.save('ratio_energies_analysis.npy', energies);

## Plotting

In [ ]:
results.ion.load('0001.ion')
plot_2d_simu(ion, results.ion.current_df, results.crystal.dataframes['0000.cfg'], radii, colors, start = True, xlim = (-20, 20), ylim = (-15, 30))

In [ ]:
df = results.ion.current_df

In [ ]:
sns.lineplot(data = df, x = 't', y = 'ke/E_i')

In [ ]:
sns.lineplot(data = df, x = 't', y = 'ipe')

In [ ]:
sns.lineplot(data = df, x = 't', y = 'epe') # external potential energy